# Mapeamento de patentes

## Bibliotecas

In [ ]:
import pandas as pd
import datetime

## Limpeza

In [ ]:
# Exporte o DataFrame para o arquivo Excel
df = pd.read_excel('data_patentes_excel.xlsx')
# Crie um DataFrame com a coluna 'ID'
df.insert(0, 'ID', ["ID{:05d}".format(i) for i in range(1, len(df) + 1)])
df.head()

In [ ]:
df_titular = df[['ID', 'titular_nome']]
df_titular.head()
df_titular.info()

In [ ]:
df_titular.loc[:, 'num_elementos'] = df_titular['titular_nome'].apply(lambda x: len(x.split(';')))
max = df_titular['num_elementos'].max()

print(max)


In [ ]:
df_titular[["titular{:02d}".format(i) for i in range(1, 36)]] = df_titular['titular_nome'].str.split(';', expand=True)

In [ ]:
titulares_melted = pd.melt(df_titular, 
                    id_vars=['ID'], 
                    value_vars=["titular{:02d}".format(i) for i in range(1, 36)], 
                    var_name='ordem_titularid', 
                    value_name='titulares')

titulares_melted = titulares_melted.dropna(subset= ['titulares'])
titulares_melted = titulares_melted.sort_values('ID')

def criar_nova_variavel(row):
    if row['ordem_titularid'] == 'titular01':
        return 'Titular'
    else:
        return 'Parceiro'
    
titulares_melted['parceria'] = titulares_melted.apply(criar_nova_variavel, axis=1)
titulares_melted['titulares'] = titulares_melted['titulares'].str.strip()

titulares_melted.head(10)


In [ ]:
titulares_melted.to_excel('titulares_melted.xlsx', sheet_name='Parcerias', index=False)

In [ ]:
tab = titulares_melted.groupby('ID').count()
n = tab.groupby('titulares').count().reset_index()
n['porcentagem'] = round((n['ordem_titularid'] / n['ordem_titularid'].sum()) * 100,2)

print(n)